In [ ]:
# Imports

import numpy as np
from kanon.tables import HTable
from kanon.units import Sexagesimal
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u

In [ ]:
# 3 Juillet 1327

year = 1327
month = 7
day = 3

In [ ]:
# Calcul du jour

month_days = [31,59,90,120,151,181,212,243,273,304,334,365]

def qtsp_from_date(y, m, d):
    ndays = int((year-1) * 365.25 + month_days[month-2] + (day - 1))
    q, p = divmod(ndays, 60)
    q, s = divmod(q, 60)
    q, t = divmod(q, 60)
    return (q, t, s, p)

days = qtsp_from_date(year, month, day)
days

In [ ]:
# Définition d'une fonction pour calculer une position à partir d'une table de mouvement moyen

def position_from_table(qtsp, tab, radix, zodiac_offset=4, width=9):
    # Additions à partir de la racine

    result = radix

    # Additions à partir des jours
    with set_precision(pmode=PrecisionMode.MAX):
        for i, v in enumerate(qtsp):
            result += tab.get(v) >> i + zodiac_offset

    # Conversion en quantité degrés avec signe zodiaque

    result %= Sexagesimal(6, 0) * u.degree
    return result

In [ ]:
# Position moyenne du soleil

# Lecture de la table depuis DISHAS
tab_mean_motion = HTable.read(193, format="dishas")
tab_mean_motion

In [ ]:
# Calcul de la position moyenne à partir des jours, de la table, et de la racine

mean_sun_pos = position_from_table(days, tab_mean_motion, Sexagesimal("4,38;21,0,30,28") * u.degree)
mean_sun_pos

In [ ]:
# Étoiles fixes

tab_fixed_stars = HTable.read(236, format="dishas")

mean_fixed_star_pos = position_from_table(days, tab_fixed_stars, 0)
mean_fixed_star_pos

In [ ]:
# Accès et recès, auge

tab_access_recess = HTable.read(237, format="dishas")

access_recess_pos = position_from_table(days, tab_access_recess, Sexagesimal("5,59;12,34")*u.degree, zodiac_offset=2, width=7)
access_recess_pos

In [ ]:
# Équation accès et recès

tab_eq_access_recess = HTable.read(238, format="dishas")

with set_precision(pmode=PrecisionMode.MAX, tmode=TruncatureMode.ROUND):
    eq_access_recess = tab_eq_access_recess.get(access_recess_pos.value)
eq_access_recess

In [ ]:
# Apogée du soleil
with set_precision(pmode=PrecisionMode.MAX, tmode=TruncatureMode.ROUND):
    solar_apogee_pos = mean_fixed_star_pos + eq_access_recess + Sexagesimal("1,11;25,23") * u.degree
solar_apogee_pos

In [ ]:
# Argument moyen du soleil
with set_precision(pmode=PrecisionMode.MAX, tmode=TruncatureMode.ROUND):
    mean_arg_sun = mean_sun_pos + (Sexagesimal(6,0) * u.degree if mean_sun_pos < solar_apogee_pos else 0) - solar_apogee_pos
mean_arg_sun

In [ ]:
# Équation du soleil

tab_eq_sun = HTable.read(19, format="dishas")
with set_precision(pmode=PrecisionMode.MAX, tmode=TruncatureMode.ROUND):
    eq_sun = tab_eq_sun.get(mean_arg_sun.value)
eq_sun

In [ ]:
# Vraie position du soleil

with set_precision(pmode=PrecisionMode.CUSTOM, tmode=TruncatureMode.ROUND, custom_precision=2):
    true_pos_sun = mean_sun_pos + (Sexagesimal(6,0) * u.degree if mean_sun_pos < eq_sun else 0) - eq_sun
true_pos_sun